In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# 1. Load the data
df_video_game = pd.read_csv("../data/clean/video_game_sales_final_cleaned.csv")
df_video_game.tail()

,name,platform,year_of_release,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,rating,rating_grouped,name_clean
59374,world end syndrome,PS4,2018,visual novel,Arc System Works,0.0,0.0,0.0,0.0,0.0,NaN,NaN,world end syndrome
59375,xblaze lost: memories,PC,2016,visual novel,Aksys Games,0.0,0.0,0.0,0.0,0.0,NaN,NaN,xblaze lost memories
59376,"yoru, tomosu",PS4,2020,visual novel,Nippon Ichi Software,0.0,0.0,0.0,0.0,0.0,NaN,NaN,yoru tomosu
59377,"yoru, tomosu",NS,2020,visual novel,Nippon Ichi Software,0.0,0.0,0.0,0.0,0.0,NaN,NaN,yoru tomosu
59378,yunohana spring! ~mellow times~,NS,2019,visual novel,Idea Factory,0.0,0.0,0.0,0.0,0.0,NaN,NaN,yunohana spring mellow times


In [ ]:
df_video_game['global_sales'].describe()
#Total sales in the world (in millions of units)

In [ ]:
df_video_game.nlargest(10, 'global_sales')


In [ ]:
df_video_game['rating_grouped'].unique()

In [ ]:

# 2. Drop rows with missing target (Global_Sales)
df_video_game = df_video_game.dropna(subset=['global_sales'])

# 3. Drop unnecessary columns
df_video_game = df_video_game.drop(columns=[
    'name', 'name_clean'
])
# 4 fill nan values inrating group with unrated
df_video_game['rating_grouped'] = df_video_game['rating_grouped'].fillna('Unrated')

In [ ]:
# 5. Create binary target: was the game a "hit"?
df_video_game['Hit'] = (df_video_game['global_sales'] > 1.0).astype(int)

# 6. Define features and target
features = ['platform', 'genre', 'publisher', 'rating']
X = df_video_game[features]
y = df_video_game['Hit']

# 7. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# 8. One-hot encode categorical features
categorical_features = ['platform', 'genre', 'publisher', 'rating']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# 9. Model pipeline
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 10. Train
clf.fit(X_train, y_train)

In [ ]:
# 11. Evaluate
accuracy = clf.score(X_test, y_test)
print(f"Model Accuracy with Rating: {accuracy:.2f}")